# Exercise Sheet 6

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import decomposition
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard

In [2]:
print(tf.__version__)

2.1.0


Some general information on the `Model()` syntax can be found [here](https://keras.io/getting-started/functional-api-guide/).

## 1 U-Net

Re-implementing networks which are discussed in the literature is a vital skill. Here you re-build the architecture from [arXiv:1505.04597](https://arxiv.org/abs/1505.04597). A figure of the network is shown in Figure 1 of this paper.  
You can check your results again via `model.compile()` and `model.summary()`.
<img src="net.png" style="width: 600px;"/>

### Solution

In [11]:
# declare imput size
inputs = Input((572,572,1))

# define weight initializations according to what is suggested in the paper below eq. (2)
init = tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='normal', seed=None)

# first box of convolutional 3x3 layers with cropping to copy the layer later on and max pooling
# later layers basically the same
Contract1conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init) (inputs)
Contract1conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init) (Contract1conv1)

print(np.shape(Contract1conv2))
Contract1crop = Cropping2D(cropping=((88, 88), (88, 88)))(Contract1conv2)
Contract1pool = MaxPooling2D((2, 2)) (Contract1conv2)

print(np.shape(Contract1pool))

Contract2conv1 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init) (Contract1pool)
Contract2conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init) (Contract2conv1)
Contract2crop = Cropping2D(cropping=((40, 40), (40, 40)))(Contract2conv2)
Contract2pool = MaxPooling2D((2, 2)) (Contract2conv2)

Contract3conv1 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=init) (Contract2pool)
Contract3conv2 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=init) (Contract3conv1)
Contract3crop = Cropping2D(cropping=((16, 16), (16, 16)))(Contract3conv2)
Contract3pool = MaxPooling2D((2, 2)) (Contract3conv2)

Contract4conv1 = Conv2D(512, (3, 3), activation='relu', kernel_initializer=init) (Contract3pool)
Contract4conv2 = Conv2D(512, (3, 3), activation='relu', kernel_initializer=init) (Contract4conv1)

print(f"4: {np.shape(Contract4conv2)}")

Contract4crop = Cropping2D(cropping=((4, 4), (4, 4)))(Contract4conv2)
Contract4pool = MaxPooling2D(pool_size=(2, 2)) (Contract4conv2)

print(np.shape(Contract4conv2))
print(np.shape(Contract4crop))

Contract5conv1 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer=init) (Contract4pool)
Contract5conv2 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer=init) (Contract5conv1)

print(np.shape(Contract5conv2))

# transpose convolution and concatenate with contracting path
# followed by convolutions
Expand4convtrans = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same', kernel_initializer=init) (Contract5conv2)
Expand4concat = concatenate([Expand4convtrans, Contract4crop])
Expand4conv1 = Conv2D(512, (3, 3), activation='relu', kernel_initializer=init) (Expand4concat)
Expand4conv2 = Conv2D(512, (3, 3), activation='relu', kernel_initializer=init) (Expand4conv1)

Expand3convtrans = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', kernel_initializer=init) (Expand4conv2)
Expand3concat = concatenate([Expand3convtrans, Contract3crop])
Expand3conv1 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=init) (Expand3concat)
Expand3conv2 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=init) (Expand3conv1)

Expand2convtrans = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', kernel_initializer=init) (Expand3conv2)
Expand2concat = concatenate([Expand2convtrans, Contract2crop])
Expand2conv1 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init) (Expand2concat)
Expand2conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init) (Expand2conv1)

Expand1convtrans = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', kernel_initializer=init) (Expand2conv2)
Expand1concat = concatenate([Expand1convtrans, Contract1crop], axis=3)
Expand1conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init) (Expand1concat)
Expand1conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init) (Expand1conv1)

# sigmoid = softmax for two classes
outputs = Conv2D(2, (1, 1), activation='sigmoid', kernel_initializer=init) (Expand1conv2)

unet = Model(inputs=inputs, outputs=outputs)
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
unet.summary()

(None, 568, 568, 64)
(None, 284, 284, 64)
4: (None, 64, 64, 512)
(None, 64, 64, 512)
(None, 56, 56, 512)
(None, 28, 28, 1024)
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 572, 572, 1) 0                                            
__________________________________________________________________________________________________
conv2d_133 (Conv2D)             (None, 570, 570, 64) 640         input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_134 (Conv2D)             (None, 568, 568, 64) 36928       conv2d_133[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_28 (MaxPooling2D) (None, 284, 284, 64) 0           

For a neat visualization, we use tensorboard:

In [4]:
# get session graph
# graph = K.get_session().graph
graph = tf.compat.v1.keras.backend.get_session().graph

# write to file
tb_path = "logs_unet/"
# writer = tf.summary.FileWriter(logdir=tb_path, graph=graph)
writer = tf.compat.v1.summary.FileWriter(logdir=tb_path, graph=graph)
K.clear_session()

RuntimeError: tf.summary.FileWriter is not compatible with eager execution. Use tf.contrib.summary instead.

In [ ]:
# run tensorboard in shell -> interrupt kernel to stop
# you can click the link to see the graph of the network we built
!tensorboard --logdir=logs_unet --port=6006

If the above link does not work for some reason, you should find tensorboard at [localhost:6006](http://localhost:6006)